In [1]:
import torch
#import ttnn
from torch import nn

torch.manual_seed(123)

In [2]:
context = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

batch = torch.stack((context, context), dim=0)
d = context.shape
d_in = context.shape[1]
d_out = context.shape[1] - 1

d_in, d_out

(3, 2)

In [3]:

class CausalAttention(nn.Module):
  def __init__(self, d_in, d_out, context_length, dropout, qkv_bias=False):
    super().__init__()

    self.d_out = d_out
    
    self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
    
    self.dropout = nn.Dropout(dropout)

    self.mask = torch.triu(
      torch.ones(context_length, context_length),
      diagonal=1
    )

  def forward(self, x): 
    b, num_tokens, d_in = x.shape
    keys = self.W_query(x)
    queries = self.W_query(x)
    values = self.W_value(x)

    # transpose the last 2 dimensions while leaving the batch dimension alone.
    attn_scores = queries @ keys.transpose(1, 2)
    attn_scores.masked_fill(
      self.mask.bool()[:num_tokens, :num_tokens], -torch.inf
    )
    attn_weights = torch.softmax(
      attn_scores / keys.shape[-1]**0.5, dim=-1
    )

    attn_weights = self.dropout(attn_weights)

    context_vec = attn_weights @ values

    return context_vec


In [4]:
torch.manual_seed(123)
context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0.0)
ca(batch)


tensor([[[-0.5287, -0.0976],
         [-0.5293, -0.1053],
         [-0.5293, -0.1052],
         [-0.5287, -0.1072],
         [-0.5287, -0.1038],
         [-0.5288, -0.1080]],

        [[-0.5287, -0.0976],
         [-0.5293, -0.1053],
         [-0.5293, -0.1052],
         [-0.5287, -0.1072],
         [-0.5287, -0.1038],
         [-0.5288, -0.1080]]], grad_fn=<UnsafeViewBackward0>)

In [5]:
class MultiHeadAttentionWrapper(nn.Module):
  def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
    super().__init__()

    self.heads = nn.ModuleList(
      [
        CausalAttention(d_in, d_out, context_length, dropout, qkv_bias) 
        for _ in range(num_heads)
      ]
    )

  def forward(self, x):
    return torch.cat([head(x) for head in self.heads], dim=-1)



In [6]:
torch.manual_seed(123)
context_length = batch.shape[1]
mha = MultiHeadAttentionWrapper(d_in, d_out, context_length, 0.0, num_heads=2)
context_vecs = mha(batch)
context_vecs, context_vecs.shape

(tensor([[[-0.5287, -0.0976,  0.5122,  0.3448],
          [-0.5293, -0.1053,  0.5123,  0.3449],
          [-0.5293, -0.1052,  0.5121,  0.3448],
          [-0.5287, -0.1072,  0.5096,  0.3438],
          [-0.5287, -0.1038,  0.5078,  0.3427],
          [-0.5288, -0.1080,  0.5113,  0.3446]],
 
         [[-0.5287, -0.0976,  0.5122,  0.3448],
          [-0.5293, -0.1053,  0.5123,  0.3449],
          [-0.5293, -0.1052,  0.5121,  0.3448],
          [-0.5287, -0.1072,  0.5096,  0.3438],
          [-0.5287, -0.1038,  0.5078,  0.3427],
          [-0.5288, -0.1080,  0.5113,  0.3446]]], grad_fn=<CatBackward0>),
 torch.Size([2, 6, 4]))

In [15]:
class MultiHeadAttention(nn.Module):
  def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
    super().__init__()

    self.d_out = d_out
    self.num_heads = num_heads
    self.head_dim = d_out // num_heads # Reduce the projection dimension to match the output dim

    self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.out_proj = nn.Linear(d_out, d_out) # linear layer to combine head outputs
    self.dropout = nn.Dropout(dropout)

    self.mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)

  def forward(self, x):
    b, num_tokens, d_in = x.shape

    keys = self.W_key(x)
    queries = self.W_query(x)
    values = self.W_value(x)

    keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
    values = values.view(b, num_tokens, self.num_heads, self.head_dim)
    queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)
    
    keys = keys.transpose(1, 2)
    queries = queries.transpose(1, 2)
    values = values.transpose(1, 2)

    attn_scores = queries @ keys.transpose(2, 3)

    mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

    attn_scores.masked_fill_(mask_bool, -torch.inf)

    attn_weights = torch.softmax(attn_scores / keys.shape[-1] ** 0.5, dim=-1)
    attn_weights = self.dropout(attn_weights)

    context_vec = torch.transpose((attn_weights @ values), 1, 2)

    context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
    context_vec = self.out_proj(context_vec)

    return context_vec

In [16]:
torch.manual_seed(123)

batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)


tensor([[[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]],

        [[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])
